In [1]:
from pysat.formula import CNF
from pysat.solvers import Glucose3

# Constants for the problem with five houses and five categories
K = 5  # Number of houses
CATEGORIES = 5  # Categories: Nationality, Color, Drink, Pet, Cigarette
M = 5  # Number of attributes per category

# Categories and attributes
NATIONALITY, COLOR, DRINK, PET, CIGARETTE = 0, 1, 2, 3, 4
ENGLISHMAN, SPANIARD, UKRAINIAN, NORWEGIAN, JAPANESE = 0, 1, 2, 3, 4
RED, GREEN, IVORY, YELLOW, BLUE = 0, 1, 2, 3, 4
COFFEE, TEA, MILK, ORANGE_JUICE, WATER = 0, 1, 2, 3, 4
DOG, SNAILS, ZEBRA, FOX, HORSE = 0, 1, 2, 3, 4
OLD_GOLD, KOOLS, CHESTERFIELDS, LUCKY_STRIKE, PARLIAMENTS = 0, 1, 2, 3, 4

def var(house, category, attribute):
    """Compute a unique variable identifier given a house, category, and attribute."""
    return house * CATEGORIES * M + category * M + attribute + 1

def bi_implication(A, B):
    """Generates clauses for A <-> B."""
    return [[-A, B], [-B, A]]

def next_to(house, category, attribute):
    """Generates clauses for 'next to' constraints."""
    clauses = []
    if house > 0:
        clauses.append([-var(house, category, attribute), var(house - 1, category, attribute)])
    if house < K - 1:
        clauses.append([-var(house, category, attribute), var(house + 1, category, attribute)])
    return clauses

def unique_attribute():
    """Ensures each house has exactly one of each attribute and no two houses have the same attribute in the same category."""
    clauses = []
    for house in range(K):
        for category in range(CATEGORIES):
            clauses.append([var(house, category, attr) for attr in range(M)])
            for attr1 in range(M):
                for attr2 in range(attr1 + 1, M):
                    clauses.append([-var(house, category, attr1), -var(house, category, attr2)])
    
    for category in range(CATEGORIES):
        for attr in range(M):
            for house1 in range(K):
                for house2 in range(house1 + 1, K):
                    clauses.append([-var(house1, category, attr), -var(house2, category, attr)])
    return clauses

# Construct formula as a list of lists
formula = unique_attribute()

# Add the puzzle clues using helper functions
for house in range(K):
    formula.extend(bi_implication(-var(house, NATIONALITY, ENGLISHMAN), var(house, COLOR, RED)))
    formula.extend(bi_implication(-var(house, NATIONALITY, SPANIARD), var(house, PET, DOG)))
    formula.extend(bi_implication(-var(house, DRINK, COFFEE), var(house, COLOR, GREEN)))
    formula.extend(bi_implication(-var(house, CIGARETTE, OLD_GOLD), var(house, PET, SNAILS)))
    formula.extend(bi_implication(-var(house, CIGARETTE, KOOLS), var(house, COLOR, YELLOW)))
    formula.extend(bi_implication(-var(house, CIGARETTE, LUCKY_STRIKE), var(house, DRINK, ORANGE_JUICE)))
    formula.extend(bi_implication(-var(house, NATIONALITY, JAPANESE), var(house, CIGARETTE, PARLIAMENTS)))
    formula.extend(bi_implication(-var(house, NATIONALITY, UKRAINIAN), var(house, DRINK, TEA)))


for house in range(K - 1):
    formula.append([-var(house, COLOR, IVORY), var(house + 1, COLOR, GREEN)])

formula.append([var(2, DRINK, MILK)])
formula.append([var(0, NATIONALITY, NORWEGIAN)])

for house in range(K):
    formula.extend(next_to(house, CIGARETTE, CHESTERFIELDS))
    formula.extend(next_to(house, PET, FOX))
    formula.extend(next_to(house, CIGARETTE, KOOLS))
    formula.extend(next_to(house, PET, HORSE))
    formula.extend(next_to(house, NATIONALITY, NORWEGIAN))
    formula.extend(next_to(house, COLOR, BLUE))
    
solver = Glucose3(bootstrap_with=formula)
solution = solver.solve()


if solution:
    print("Solution found!")
else:
    print("No solution found.")

No solution found.


In [19]:
formula = unique_attribute()
solver = Glucose3(bootstrap_with=formula)
if not solver.solve():
    print("Basic constraints unsolvable!")
else:
    clues = [
        # The Englishman lives in the red house.
        lambda house: formula.extend(bi_implication(var(house, NATIONALITY, ENGLISHMAN), var(house, COLOR, RED))),
        lambda house: formula.extend(bi_implication(var(house, NATIONALITY, SPANIARD), var(house, PET, DOG))),
        lambda house: formula.extend(bi_implication(var(house, DRINK, COFFEE), var(house, COLOR, GREEN))),
        lambda house: formula.extend(bi_implication(var(house, NATIONALITY, UKRAINIAN), var(house, DRINK, TEA))),
        lambda house: formula.append([-var(house, COLOR, IVORY), var(house + 1, COLOR, GREEN)]) if house < K - 1 else None,
        lambda house: formula.extend(bi_implication(var(house, CIGARETTE, OLD_GOLD), var(house, PET, SNAILS))),
        lambda house: formula.extend(bi_implication(var(house, CIGARETTE, KOOLS), var(house, COLOR, YELLOW))),
        lambda house: formula.append([var(2, DRINK, MILK)]) if house == 2 else None,
        lambda house: formula.append([var(0, NATIONALITY, NORWEGIAN)]) if house == 0 else None,
        lambda house: (formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house + 1, PET, FOX)]) if house < K - 1 else None,
                    formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house - 1, PET, FOX)]) if house > 0 else None),
        lambda house: (formula.append([-var(house, CIGARETTE, KOOLS), var(house + 1, PET, HORSE)]) if house < K - 1 else None,
                    formula.append([-var(house, CIGARETTE, KOOLS), var(house - 1, PET, HORSE)]) if house > 0 else None),

        lambda house: formula.extend(bi_implication(var(house, CIGARETTE, LUCKY_STRIKE), var(house, DRINK, ORANGE_JUICE))),
        lambda house: (formula.append([-var(house, NATIONALITY, JAPANESE), var(house, CIGARETTE, PARLIAMENTS)]),
                    formula.append([var(house, NATIONALITY, JAPANESE), -var(house, CIGARETTE, PARLIAMENTS)]))









    ]
    
    for idx, add_clue in enumerate(clues):
        for house in range(K):
            add_clue(house)
        solver = Glucose3(bootstrap_with=formula)
        if not solver.solve():
            print(f"Unsolvable after adding clue {idx + 1}")
            break


Unsolvable after adding clue 13


In [22]:
def add_and_verify(clue_function, formula):
    """Add a clue to the formula and verify its solvability."""
    for house in range(K):
        clue_function(house)
    solver = Glucose3(bootstrap_with=formula)
    return solver.solve()

# Start with base constraints
formula = unique_attribute()

# List of clues
clues = [
    lambda house: formula.extend(bi_implication(var(house, NATIONALITY, SPANIARD), var(house, PET, DOG))),
    lambda house: formula.extend(bi_implication(var(house, NATIONALITY, SPANIARD), var(house, PET, DOG))),
    lambda house: formula.extend(bi_implication(var(house, NATIONALITY, UKRAINIAN), var(house, DRINK, TEA))),
    lambda house: formula.extend(bi_implication(var(house, DRINK, COFFEE), var(house, COLOR, GREEN))),
    lambda house: formula.append([-var(house, COLOR, IVORY), var(house + 1, COLOR, GREEN)]) if house < K - 1 else None,
    lambda house: formula.extend(bi_implication(var(house, CIGARETTE, OLD_GOLD), var(house, PET, SNAILS))),
    lambda house: formula.extend(bi_implication(var(house, CIGARETTE, KOOLS), var(house, COLOR, YELLOW))),
    lambda house: formula.append([var(2, DRINK, MILK)]) if house == 2 else None,
    lambda house: formula.append([var(0, NATIONALITY, NORWEGIAN)]) if house == 0 else None,
    lambda house: (formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house + 1, PET, FOX)]) if house < K - 1 else None,
                formula.append([-var(house, CIGARETTE, CHESTERFIELDS), var(house - 1, PET, FOX)]) if house > 0 else None),
    lambda house: (formula.append([-var(house, CIGARETTE, KOOLS), var(house + 1, PET, HORSE)]) if house < K - 1 else None,
                formula.append([-var(house, CIGARETTE, KOOLS), var(house - 1, PET, HORSE)]) if house > 0 else None),
    lambda house: formula.extend(bi_implication(var(house, CIGARETTE, LUCKY_STRIKE), var(house, DRINK, ORANGE_JUICE))),
    lambda house: (formula.append([-var(house, NATIONALITY, JAPANESE), var(house, CIGARETTE, PARLIAMENTS)]),
                formula.append([var(house, NATIONALITY, JAPANESE), -var(house, CIGARETTE, PARLIAMENTS)]))


]

# Encode & verify each clue
for clue in clues:
    if not add_and_verify(clue, formula):
        print(f"Unsolvable after adding clue: {clue}")


Unsolvable after adding clue: <function <lambda> at 0x7fec218560e0>
